In [4]:
def Preprocess(file):
    array = []
    for line in f:
        array.append(line)
    xc = [0]*300
    yc = [0]*300
    maxrows = 300
    maxcols = 300
    Cost = [([0] * maxcols) for row in range(maxrows) ]

    dim = int(array[0])
    for it in range(1, dim+1):
        p = array[it]
        tokens = p.split()
        xc[it-1] = float(tokens[0])
        yc[it-1] = float(tokens[1])
    
    for i in range(0, dim):
        for j in range(0, dim):
            Cost[i][j] = pow((pow(xc[i]-xc[j], 2) + pow(yc[i] - yc[j], 2)), 0.5)
        
    for i in range(0, dim):
        for j in range(0, dim):
            if(i==j):
                Cost[i][j] = 99999
    return Cost, xc, yc, dim


In [2]:

def findcomponents(x):
    component = 0
    for i in range(0, maxrows):
        if (visited[i] == 0):
            j = 0
            dfs(x, i, component, j)
            component = component +1

def dfs(M, i, l, j):
    visited[i] = 1
    components[l][j] = i
    #j = j+1
    for k in range(0, maxrows):
        if(M[i][k] == 1 and visited[k] != 1):
            dfs(M, k, l, j+1)

In [6]:
from gurobipy import *
import string
import time

f = open("C:\\Users\\Prasangsha\\Downloads\\TSP_instance_n_15_s_1.dat")
Cost, xc, yc, dim = Preprocess(f)

visited = [0]*dim
maxrows = dim
maxcols = dim

visited = [0]*dim
yc = [0]*300
maxrows = dim
maxcols = 4
#x = [([0] * dim) for row in range(dim) ]
components = [([-1] * dim) for row in range(dim) ]


TSP = Model('TSP')
import time
startTime = time.time()
x = {}
for i in range(dim):
    for j in range(dim):
        x[i,j] = TSP.addVar(vtype=GRB.BINARY, obj = Cost[i][j], name = 'x_{}_{}'.format(i,j))
            
TSP.modelSense = GRB.MINIMIZE
TSP.Params.lazyConstraints = 1
TSP.update()

for i in range(dim):
    TSP.addConstr(quicksum(x[i,j] for j in range(dim)) == 1, name='leave_{}'.format(i))
    TSP.addConstr(quicksum(x[j,i] for j in range(dim)) == 1, name='enter_{}'.format(i))

# Load data into the model
TSP._x = x
TSP._n = dim

def subtour(model, where):
    if where == GRB.Callback.MIPSOL:
        # Retrieve Current Solution
        xoy = [[0 for i in range(dim)] for j in range(dim)]
        x_sol = model.cbGetSolution(model._x)
        for i,j in model._x.keys(): 
            if (x_sol[i,j] > 0):
                xoy[i][j] = 1
        for i in range(0, model._n):
            visited[i] = 0
        for i in range(0, model._n):
            for j in range(0, model._n):
                components[i][j] = -1
        findcomponents(xoy)
        number_comp = 0
        for l in range(0, model._n):
            if(components[l][0] != -1):
                number_comp = number_comp+1
        if(number_comp > 1):
            for l in range(0, number_comp):
                len_c = 0
                temp = [-1]*model._n
                for j in range(0, model._n):
                    if (components[l][j] != -1):
                        temp[j] = components[l][j]
                        len_c = len_c +1
                model.cbLazy(quicksum(x[i,k] for i in temp for k in temp if (i !=k and i != -1 and k != -1 )) <= len_c - 1)


TSP.optimize(subtour)
executionTime = (time.time() - startTime)
print("The execution time is")
print(executionTime)



Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 30 rows, 225 columns and 450 nonzeros
Model fingerprint: 0xa5dcbba3
Variable types: 0 continuous, 225 integer (225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 30 rows, 225 columns, 450 nonzeros
Variable types: 0 continuous, 225 integer (225 binary)

Root relaxation: objective 5.967075e+01, 36 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      65.1316120   65.13161  0.00%     -    0s

Cutting planes:
  Lazy constraints: 23

Explored 0 nodes (59 simplex iterations) in 0.03 seconds
Thread count was 8 (of 8 available p